In [5]:
from PIL import Image, ImageDraw, ImageFont
import os
import json
from skimage.metrics import structural_similarity as ssim
import numpy as np

In [7]:
dataset_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\sewfactory'
save_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\Paula_dataset'

def get_70_70_images(name):
    good_dresses = []
    images = []
    for folder_dress in os.listdir(dataset_path):
        if folder_dress.startswith(name):
                try:
                    image = Image.open(os.path.join(dataset_path, folder_dress, 'static/static_0_30.png'))
                    # resize to 20x20
                    image = image.resize((70, 70))
                    # if pixel is not white, make it black
                    for i in range(image.size[0]):
                        for j in range(image.size[1]):
                            if image.getpixel((i, j)) != (255, 255, 255):
                                image.putpixel((i, j), (0, 0, 0))
                    image = image.convert('L')
                    image_np = np.array(image)
                    images.append(image_np)
                    #image.save(os.path.join(save_path, folder_dress+'.png'))
                    good_dresses.append(folder_dress)
                except:
                    print('error on ', folder_dress)
                    continue

    return images, good_dresses

images, good_dresses = get_70_70_images('dress')


error on  dress_sleeveless_7716FF7QWP


In [8]:
def find_uniques(unique_images):
    counter=0
    threshold = 0.98
    n = len(unique_images)
    similar_pairs_unique = []
    for i in range(n):
        index_i = good_dresses.index(unique_images[i])
        for j in range(i + 1, n):
            index_j =  good_dresses.index(unique_images[j])
            sim = ssim(images[index_i], images[index_j])
            if sim > threshold:
                #print(good_dresses[i], good_dresses[j], sim)
                counter = counter + 1
                similar_pairs_unique.append((unique_images[i], unique_images[j], sim))
                
        if i%100 == 0:
            print("indice:", i, "pares encontrados:", counter)

    unique_images = []
    for i in range(len(similar_pairs_unique)):
        unique_images.append(similar_pairs_unique[i][0])

    unique_images = np.unique(unique_images)
    print("----------------------------\n"
          "----------------------------\n"
          "----------------------------\n")
    print(len(unique_images))
    return unique_images



unique_images = good_dresses.copy()
unique_images = find_uniques(unique_images)

indice: 0 pares encontrados: 17
indice: 100 pares encontrados: 206
indice: 200 pares encontrados: 319
indice: 300 pares encontrados: 461
indice: 400 pares encontrados: 582
indice: 500 pares encontrados: 697
indice: 600 pares encontrados: 790
indice: 700 pares encontrados: 886
indice: 800 pares encontrados: 1040
indice: 900 pares encontrados: 1121
indice: 1000 pares encontrados: 1237
indice: 1100 pares encontrados: 1335
indice: 1200 pares encontrados: 1391
indice: 1300 pares encontrados: 1431
indice: 1400 pares encontrados: 1487
indice: 1500 pares encontrados: 1526
indice: 1600 pares encontrados: 1551
indice: 1700 pares encontrados: 1597
indice: 1800 pares encontrados: 1631
indice: 1900 pares encontrados: 1656
indice: 2000 pares encontrados: 1699
indice: 2100 pares encontrados: 1724
indice: 2200 pares encontrados: 1737
indice: 2300 pares encontrados: 1750
indice: 2400 pares encontrados: 1758
indice: 2500 pares encontrados: 1760
----------------------------
----------------------------
-

In [9]:
len(unique_images)

482

In [11]:
unique_images_v2 = find_uniques(unique_images_v2)

indice: 0 pares encontrados: 14
indice: 100 pares encontrados: 416
indice: 200 pares encontrados: 834
indice: 300 pares encontrados: 1002
----------------------------
----------------------------
----------------------------

246


In [12]:
def save_unique_images(unique_images): 
    save_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\Paula_dataset_unique'

    for folder_dress in unique_images:
        image = Image.open(os.path.join(dataset_path, folder_dress, 'static/static_0_30.png'))    
        image = image.convert('L')
        image.save(os.path.join(save_path, folder_dress+'.png'))

In [13]:
save_unique_images(unique_images_v2)

In [85]:
with open('unique_atributes.txt', 'r') as file_unique:
    for line in file_unique:
        print(line)

Tamaño de manga: 

    - Ancha

    - Estrecha



Largo de manga:

    - Sisa

    - Corta

    - Al hombro

  

Cuello: 

    - En V

    - Redondo



Largo de falda: 

    - Larga 

    - Corta

    - Midi



Ancho de falda:

    - Ancha

    - Pegada



In [20]:
def process_coords(coordinates, size = 250):
    for coord in coordinates:
        coord[0] = int(-3.5*coord[0] + size)
        coord[1] = int(-3.5*coord[1] + size)
    return coordinates

def distance_between_points(p1, p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5


def save_coordinates(folder, savepath):

    prueba_path = "P:/Documentos/HDSP/Fashion_garments_measurements/dataset/sewfactory/"+folder+"/static/"+folder+"_specification.json"

    with open(prueba_path, 'r') as file:
        data = json.load(file)

    for dress_part in ["top_front", "top_back", "skirt_front", "skirt_back"]:

        coordinates = data['pattern']['panels'][dress_part]['vertices']
        coordinates = process_coords(coordinates)
        font = ImageFont.truetype("arial.ttf", 16)

        # new image with white background
        img = Image.new('RGB', (500, 500), color = (255, 255, 255))

        # draw the first two dots 
        draw = ImageDraw.Draw(img)
        draw.ellipse((coordinates[0][0]-2, coordinates[0][1]-2, coordinates[0][0]+2, coordinates[0][1]+2), fill=(0,0,0), outline=(0,0,0))
        draw.ellipse((coordinates[1][0]-2, coordinates[1][1]-2, coordinates[1][0]+2, coordinates[1][1]+2), fill=(0,0,0), outline=(0,0,0))

        Y = []

        with open(savepath+'/'+dress_part+'_distances.txt', 'w') as file:
            for i in range(len(coordinates)): 
                first_y = coordinates[0][1]
            
                if coordinates[i][1] == coordinates[(i+1)%len(coordinates)][1]:
                    draw.line((coordinates[i][0], coordinates[i][1], coordinates[(i+1)%len(coordinates)][0], coordinates[(i+1)%len(coordinates)][1]), fill=(255,0,0), width=2)
                    Y.append(coordinates[i][1])
                else:
                    draw.line((coordinates[i][0], coordinates[i][1], coordinates[(i+1)%len(coordinates)][0], coordinates[(i+1)%len(coordinates)][1]), fill=(0,0,0), width=2)
                draw.text((coordinates[i][0], coordinates[i][1]), str(i), (0,0,0), font=font)

                file.write("[" + str(i) + "," + str((i+1)%len(coordinates))+"]: "
                        +str(distance_between_points(coordinates[i], coordinates[(i+1)%len(coordinates)]))+'\n')

            draw.line((250, first_y, 250, Y[1]), fill=(0,255,0), width=2)
            draw.text((250, first_y), str(len(coordinates)+1), (0,0,0), font=font)
            draw.text((250, Y[1]), str(len(coordinates)+2), (0,0,0), font=font)
            file.write("[" + str(len(coordinates)+1) + "," + str(len(coordinates)+2)+ "]: "+str(distance_between_points([250, first_y], [250, Y[1]]))+'\n')

        draw = ImageDraw.Draw(img)
        draw.text((0, 0), dress_part, (0,0,0), font=font)

        img.save(os.path.join(savepath, dress_part+'_points_coordinates.png'))

In [21]:
new_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\Paula_dataset_unique'
import shutil

dataset_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\sewfactory'
save_path = 'P:\Documentos\HDSP\Fashion_garments_measurements\dataset\Clean_dataset'



for file in os.listdir(new_path):
    if file.endswith('.png'):
        print(file)
        folder = file.split('.')[0]
        txt_file = file.replace('.png', '_atributes.txt')
        if not os.path.exists(os.path.join(save_path, folder)):
            os.mkdir(os.path.join(save_path, folder))
        # copiar txt de los atributos de la prenda
        shutil.copy(os.path.join(new_path, txt_file), os.path.join(save_path, folder, txt_file))

        # copiar imagenes de la prenda
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_0_30.png'), os.path.join(save_path, folder, folder+'_static_0_30.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_30_0.png'), os.path.join(save_path, folder, folder+'_static_30_0.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_60_0.png'), os.path.join(save_path, folder, folder+'_static_60_0.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_120_0.png'), os.path.join(save_path, folder, folder+'_static_120_0.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_150_0.png'), os.path.join(save_path, folder, folder+'_static_150_0.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_210_0.png'), os.path.join(save_path, folder, folder+'_static_210_0.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_240_0.png'), os.path.join(save_path, folder, folder+'_static_240_0.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_300_0.png'), os.path.join(save_path, folder, folder+'_static_300_0.png'))
        shutil.copy(os.path.join(dataset_path, folder ,'static/static_330_0.png'), os.path.join(save_path, folder, folder+'_static_330_0.png'))

        shutil.copy(os.path.join(dataset_path, folder ,'static'+folder+'_'+folder+'_uv.png'), os.path.join(save_path, folder, folder+'_uv.png'))

        save_coordinates(folder, save_path+'/'+folder)



dress_sleeveless_00F3PNYG6L.png
dress_sleeveless_013LZJLUQ0.png
dress_sleeveless_02OIQPCOWU.png
dress_sleeveless_04HEL5MHFC.png
dress_sleeveless_0IHJZ81F9V.png
dress_sleeveless_0NQV4UR4G0.png
dress_sleeveless_0PW8HGG42W.png
dress_sleeveless_0R0VSXFOXW.png
dress_sleeveless_0Z54DUIP2N.png
dress_sleeveless_1DD7UEQN0L.png
dress_sleeveless_1E8PI9A2DX.png
dress_sleeveless_1M10E9IVBQ.png
dress_sleeveless_1UHTWRK2NI.png
dress_sleeveless_3Z2PED6ZZX.png
dress_sleeveless_4CB7HGSP32.png
dress_sleeveless_4IB8MCTXX9.png
dress_sleeveless_4JIINXQRT6.png
dress_sleeveless_4U2QZ6FAMH.png
dress_sleeveless_5CS1KB1QUB.png
dress_sleeveless_5HY9KKTT6W.png
dress_sleeveless_5SQV9BLKBD.png
dress_sleeveless_65I912EJNB.png
dress_sleeveless_6BIU6715BK.png
dress_sleeveless_6D4RA1FJMJ.png
dress_sleeveless_6IVSIA6XEN.png
dress_sleeveless_6ZOSV66T3F.png
dress_sleeveless_72XWDNTO99.png
dress_sleeveless_76JKKFORV9.png
dress_sleeveless_7BA0B946DP.png
dress_sleeveless_7EWI19AX5S.png
dress_sleeveless_9GADOV728A.png
dress_sl

Tamaño de manga
1. Ancha
2. Estrecha
['Tamaño de manga: Ancha\n']
Largo de manga
1. Sisa
2. Corta
3. Al hombro
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n']
Cuello
1. En V
2. Redondo
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n', 'Cuello: Redondo\n']
Largo de falda
1. Larga
2. Corta
3. Midi
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n', 'Cuello: Redondo\n', 'Largo de falda: Larga\n']
Ancho de falda
1. Ancha
2. Pegada
['Tamaño de manga: Ancha\n', 'Largo de manga: Corta\n', 'Cuello: Redondo\n', 'Largo de falda: Larga\n', 'Ancho de falda: Pegada\n']
Tamaño de manga
1. Ancha
2. Estrecha


KeyboardInterrupt: Interrupted by user